In [ ]:
from lxml import etree
import pandas as pd
import re


file_path = "Nashville.net.xml"  
tree = etree.parse(file_path)
root = tree.getroot()

# Get all non-internal edge IDs
sumoID_org = []
for edge in root.findall("edge"):
    if edge.get("function") != "internal":
        sumoID_org.append(edge.get("id"))

# Build replacement lists
oldedgeID = []
newedgeID = []

for eid in sumoID_org:
    if not eid.startswith("-"):
        oldedgeID.append(eid)
        new_id = "-" + eid
        if new_id in sumoID_org:
            new_id = "-" + new_id
        newedgeID.append(new_id)

edgeID_replacement = pd.DataFrame({
    "oldedgeID": oldedgeID,
    "newedgeID": newedgeID
})

with open(file_path, "r", encoding="utf-8") as f:
    original_text = f.read()

# Perform replacements in the file content
for _, row in edgeID_replacement.iterrows():
    old_id = row["oldedgeID"]
    new_id = row["newedgeID"]
    original_text = re.sub(f'"{old_id}"', f'"{new_id}"', original_text)
    original_text = re.sub(f'lane id="{old_id}_', f'lane id="{new_id}_', original_text)
    original_text = re.sub(f' {old_id}_', f' {new_id}_', original_text)

with open(file_path, "w", encoding="utf-8") as f:
    f.write(original_text)




In [7]:
import subprocess
network_name = "Nashville"

cmd = [
    "netconvert",
    "-s", f"{network_name}.net.xml",
    "--opendrive-output", f"{network_name}.xodr",
    "--output.original-names", "true",
    "--junctions.scurve-stretch", "1.0"
]
process = subprocess.Popen(cmd, shell=True)
process.wait()

0